In [ ]:
# python imports
import os
import time
import functools

import numpy as np
from scipy import stats
from scipy.stats import uniform, norm, t as tdist, ttest_ind


In [ ]:
# setup graphs
%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as tkr
from matplotlib import rc

plt.rcParams['figure.figsize'] = 8, 3.5
plt.rcParams['figure.facecolor'] = 'white'
# plt.rcParams['figure.dpi'] = 100;

# numpy print format
np.set_printoptions(linewidth=120, precision=4, suppress=False, formatter={'float': '{:5.3e}'.format})

In [ ]:
# local imports
import util.tests as tests
import util.dwdb_reader as io
import util.func as f


## Load traces and classification

In [ ]:
# Read first sample point, split by classification (2 traces, c1 and c2)
print("Reading traces")
dsr = io.dwdb_reader(r'FP_db_small/log.dwdb', r'FP_db_small')
data_batch, meta_batch = dsr.read_batch(10)
print("Done")

pt_range = slice(2,4)
data_batch2, meta_batch2 = data_batch[pt_range], meta_batch[pt_range]

In [ ]:
# Convert traces to numpy
traces_np = np.asarray(data_batch2)
traces_np.shape

In [ ]:
# Extract classifiers, convert to numpy
meta_prep = [m['other'].split() for m in meta_batch2]
classifiers = [s.split('=')[1] for m in meta_prep for s in m if s.startswith('s=')]
classifiers = [str(int(c)-1) for c in classifiers]
classifiers_np = np.asarray(classifiers)
classifiers_np

In [ ]:
# Postprocessing
# 1. 2 traces should be transposed and merged in 1 sample point
traces_np = traces_np.reshape(-1).reshape(2000, 1)
# 2. classifiers should be stretched as the 1st half of the sample point is R1 and 2nd one is R2
classifiers_np = np.repeat(classifiers_np, len(traces_np)/2).reshape(2000, 1)
# traces_np[:4], classifiers_np[:4]
traces_np.shape, classifiers_np.shape

In [ ]:
small_value = 1e-300
tvla_thrd = 4.5 * np.sqrt(2)

PROJECT_ROOT_DIR=os.getcwd()
tracenum = 50*1000
step = 50
sample_start = 120 # 30  100    # 40
sample_end =   130 # 50  500    # 180
rlen = sample_end - sample_start

IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
if not os.path.exists(IMAGES_PATH): os.makedirs(IMAGES_PATH)

LOG_DIR = os.path.join(PROJECT_ROOT_DIR, "log")
if not os.path.exists(LOG_DIR): os.makedirs(LOG_DIR)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Sanity check. Make sure it leaks

In [ ]:
atrace = np.mean(traces_np[:10], axis=0)
tt = tests.fvr_ttest(traces_np, classifiers_np)
tt

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_ylabel('voltage', color='g')
ax1.tick_params(axis='y', labelcolor='g')
ax2 = ax1.twinx()
ax2.set_ylabel('leakage', color='r')
ax2.tick_params(axis='y', labelcolor='r')
ax1.plot(atrace, color='g') # signal
ax2.plot(tt, color='r')     # leak
plt.show()

In [1]:
ALL DONE TILL THIS POINT

SyntaxError: invalid syntax (<ipython-input-1-6c7c5910d8ec>, line 1)

## Evolution

In [ ]:
tstat_evo = []
cnt = 1
xx = np.arange(step, tracenum+1, step)
for x in xx:
    start = time.time()
    tt = tests.fvr_ttest(data_np[:x], classifiers_np[:x])
    if cnt % 200 == 0:
        print("Trace count: {}, done in {}".format(x, time.time() - start))
    cnt += 1
    tstat_evo.append(tt)
tstat_evo = np.array(tstat_evo)
tstat_evo = np.abs(tstat_evo) # comment out for intermediates

# Max leakage info
tr_num, max_leak_pt = np.unravel_index(np.argmax(tstat_evo), tstat_evo.shape)
max_leak_val, max_leak_at = tstat_evo[tr_num, max_leak_pt], tr_num*step
max_leak_pv = tdist.sf(max_leak_val, max_leak_at)
print("Max leak at point: {}, value: {:5.3e} (pv: {:5.3e}), trace: {}".format(
    max_leak_pt, max_leak_val, max_leak_pv, max_leak_at))

In [ ]:
step_num, pt_num = np.shape(tstat_evo)

fig = plt.figure()
plt.ylabel('T score')
plt.xlabel('Trace Number')
plt.axhline(y=tvla_thrd, color='r', linestyle=':')

x_axis = np.arange(step_num) * step

# Plot leak history for each 2nd point
for j in range(0, pt_num, 2):
    plk = tstat_evo.T[j]
    plt.plot(x_axis, plk, linewidth=0.5, linestyle='-', color = 'grey', zorder = j)

# Max leak point
mlpt = max_leak_pt
plt.plot(x_axis, tstat_evo[:, mlpt], linewidth=1, linestyle='-', color = 'r', zorder=255)
plt.show()

## Bootstrapping

In [ ]:
boots_list = [11, 21, 51]  # boots
boot_step = 200   # step
upper_bnd = 250 #tracenum
ld_stat = stats.uniform
ld_cdf = ld_stat().cdf

t_full_per_step = []   # cannot be numpy array due to different lengths per boot
tp_full_per_step = []  # cannot be numpy array due to different lengths per boot
ksp_per_step = []
for rangenum in range(boot_step, upper_bnd+1, boot_step):
    # Run all boots for a step
    t_full_per_boot, tp_full_per_boot, ksp_per_boot = [], [], []
    for j, boots in enumerate(boots_list):
        start = time.time()
        crossed_ml = 0
        t_full = np.empty((boots, rlen), dtype=np.float64)
        tp_full = np.empty_like(t_full)
        ks_full = np.empty_like(tp_full[0])
        boot_idxs = np.random.randint(rangenum, size=(boots, rangenum))
        for i, bi in enumerate(boot_idxs):
            t_full[i] = tests.fvr_ttest(data_np[bi], classifiers_np[bi]) # calc tt and keep it
            tp_full[i] = tdist.sf(t_full[i], rangenum)                   # convert tt to pv and keep pv
        # Run ks-test for p-values as the post process boot for the step
        for i in range(rlen):
            d, ks_full[i] = f.kstest(tp_full.T[i], ld_cdf)               # throw away d, keep pv
        t_full_per_boot.append(t_full)
        tp_full_per_boot.append(tp_full)
        ksp_per_boot.append(ks_full)
#         lfound = 'SAME LEAK ACHIEVED!' if ks_full[max_leak_pt] < max_leak_pv else ''
#         print("Trace count: {}, boots: {} are done in {} ({})".format(
#             rangenum, boots, time.time() - start, lfound))
    t_full_per_step.append(t_full_per_boot)
    tp_full_per_step.append(tp_full_per_boot)
    ksp_per_step.append(ksp_per_boot)
ksp_per_step = np.asarray(ksp_per_step)


In [ ]:
print(ksp_per_step)

# Bootstrapping vizualization

In [ ]:
mlpi =  max_leak_pt # 15

# Prepare data
ksp_per_step[np.where(np.isnan(ksp_per_step))] = small_value
ksp_per_step[np.where(ksp_per_step < small_value)] = small_value

pt_num, step_num = np.shape(ksp_per_step[:, 0].T)

x_axis = (np.arange(step_num) + 1) * boot_step

fig = plt.figure()
plt.xlabel('Trace Number')
plt.ylabel('ks: -log10(pv)')
# tvla_high threshold converted to pv scale
tvla_thrd_pvlog = -np.log10(max_leak_pv) # 200 is approx n
plt.axhline(y=tvla_thrd_pvlog, color='r', linestyle=':')

# Plot boot evolution for each 2nd point of 1st boot in the list (for comparison)
boot_idx = 0
plot_evo = ksp_per_step[:, boot_idx].T
for j in range(0, pt_num, 2):
    kspl = -np.log10(plot_evo[j])
    plt.plot(x_axis, kspl, linewidth=0.75, color = 'grey', zorder = j)

# and hist for max leaky point per boot
cpalette = ['b', 'y', 'g', 'r', 'c']
for i in range(ksp_per_step.shape[1]):
    maxkslpv = -np.log10(ksp_per_step[:, i].T[mlpi])  # boots_list[0]
    plt.plot(x_axis, maxkslpv, linewidth=1.5, color = cpalette[i%5], zorder=255)

plt.show()
# plot_evo.shape, x_axis.shape

## Outcome

- Real threshold is 4.5 * sqrt(2) is similar to twice as much traces (TVLA requires two runs). 
- It means that leak is not occasional
- The threshold get converted to t-distribution p-value to compare with ks-test p-value
- We can compare (number_of_traces1) required to reach the point of the real leakage with (number_of_boots) * (number_of_traces2). Where number_of_traces1 is the number for the regular TVLA process and (number_of_traces2) is the number required for bootstrapping.
  - For example TVLA requires 43.5k traces to reach 6 sigma, bootstrapping requires 225 traces * 50 boots = 11250, i.e. bootstrapping reaches the same leak *4 times faster*

# Below are cells to debug code snippets

## Debugging ks-test 

In [ ]:
# Find out whether it follows the uniform distribution
ksp_per_step2 = np.zeros_like(tp_full_per_step[:,1])

for s, bps in enumerate(tp_full_per_step):          # step
    bpsr = bps.T                                    # -> (pt, boot)
    start = time.time()
    print('{}: Step {} ...'.format(bpsr.shape, s))
    for i, ptbt in enumerate(bpsr):
        d, kpv = f.kstest(ptbt, ld_cdf)
        ksp_per_step2[s, i] = kpv
    print("Done in {}".format(time.time() - start))


## Debugging distributions for kstest

In [ ]:

mpi = max_leak_pt # 6 178
si = 2

# bpvs = t_full_per_step[si].T  # t
bpvs = tp_full_per_step[si].T   # t-pv

kpvs = []
leak_dist = stats.uniform
# leak_dist = stats.norm
for j in range(0, len(bpvs)):
    bp = bpvs[j]
    d, pv = f.kstest(bp, leak_dist().cdf)
    l, s = leak_dist.fit(bp)      # loc and scale of the dist at the point
    leaking = pv < 0.05           # 3 sigma assurance the point is leaking (non uniform)
    kpvs.append([d, pv, l, s, leaking])
d, pv = f.kstest(bpvs[mpi], leak_dist().cdf)
l, s = leak_dist.fit(bpvs[mpi])  # loc and scale of the dist at the max leak point
leaking = pv < 0.05
kpvs.append([d, pv, l, s, leaking])

# print("d, pv, l, s, leaking")
# print(np.array(kpvs))
    
# norm_cdf = norm(loc=0, scale=1).cdf
# dt, pv  = stats.kstest(bpvs, norm_cdf)

fig = plt.figure()
plt.xlabel('Boot Number')
for j in range(0, len(bpvs), 20):
    plt.plot(bpvs[j], color = 'grey', linewidth=0.5, zorder = j)

plt.plot(bpvs[mpi], color = 'r', linewidth=2, zorder=255)
plt.show()

## Debugging at some boot

In [ ]:
%%script false    # This line is to skip this cell as a whole in the real run. Comment to debug a boot step
# rp_full_per_step = []  # rpv
# t_full_per_step = []   # t
# tp_full_per_step = []  # tpv

pb = 0
bootidx = 15

pb = 0
pi = 19 #byte_pt[pb]
bi = byte_idx[pb]
hwi = key_idx[pb]   # kpv = 0

plot_evo = t_full_per_step[:,bootidx,:,pi].reshape(-1, 3, 256)

x_axis = (np.arange(len(plot_evo)) + 1) * step

fig = plt.figure()
plt.xlabel('Trace Number')
plt.ylabel('T score')

hw_num = plot_evo.shape[2]
for j in range(0, hw_num, 4):
    mtt = plot_evo[:, byte_idx[pb], j]
    plt.plot(x_axis, mtt, linewidth=0.5, color = 'grey', zorder = j)
plt.plot(x_axis, plot_evo[:, byte_idx[pb], key_idx[pb]], linewidth=1, color = 'r', zorder=255)

plt.show()


## Getting familiar with tests 

In [ ]:
ud = uniform(loc=0, scale=1)
for i in range(3):
    udrv = ud.rvs(size=1000)
    dk, kpv = stats.kstest(udrv, 'uniform') # H0 - data is uniform
    non_uniform = kpv < 0.05
    print(dk, kpv, non_uniform)

In [ ]:
x_100 = norm.rvs(loc=10, scale=1, size=100)
nd = norm(loc=0, scale=1)
for i in range(3):
    ndrv = nd.rvs(size=100)
    dn, npv = stats.kstest(ndrv, 'norm') # H0 - data is norm
    non_norm = npv < 0.05
    print(dn, npv, non_norm)

In [ ]:
nd_cdf = norm(loc=10, scale=1).cdf
for i in range(3):
    ndrv = norm.rvs(loc=10, scale=1, size=100)
    dn, npv = stats.kstest(ndrv, nd_cdf) # H0 - data is norm
    non_norm = npv < 0.05
    print(dn, npv, non_norm)